In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib

data_directory = '../data/'
file_path = data_directory + 'sample_data.csv'



In [ ]:
steemit = pd.read_csv(file_path)
steemit = steemit.ix[steemit['body'].notnull(),:]
y = steemit['total_payout_value']

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(steemit['body'])


In [ ]:
steemit = pd.DataFrame(X.toarray())
steemit.columns = vectorizer.get_feature_names()


In [ ]:
word_count = steemit.sum(axis = 1)

In [ ]:
import seaborn as sns

In [ ]:
%matplotlib inline 
sns.regplot(word_count,y, x_jitter=0.1, y_jitter=0.1)